In [148]:
import pandas as pd

from skmultiflow.meta import AdditiveExpertEnsembleClassifier
from skmultiflow.meta import DynamicWeightedMajorityClassifier
from skmultiflow.meta import LearnPPNSEClassifier


from skmultiflow.data.stagger_generator import STAGGERGenerator

import matplotlib.pyplot as plt

In [149]:
def score_stagger_after(model, n_batch, batch_size):
    correct_cnt = 0
    stream = STAGGERGenerator(classification_function = 0, balance_classes = True)
    for i in range(n_batch):
        stream.generate_drift()
        X, y = stream.next_sample(batch_size)
        
        y_pred = model.predict(X)
        correct_cnt += sum(y == y_pred)
        
        X, y = stream.next_sample(batch_size)
        model.partial_fit(X, y, classes=[0, 1])
    return correct_cnt/(n_batch*batch_size)

In [150]:
def score_stagger_before(model, n_batch, batch_size):
    correct_cnt = 0
    stream = STAGGERGenerator(classification_function = 0, balance_classes = True)
    for i in range(n_batch):
        stream.generate_drift()
        X, y = stream.next_sample(batch_size)
        model.partial_fit(X, y, classes=[0, 1])
        
        X, y = stream.next_sample(batch_size)
        y_pred = model.predict(X)
        correct_cnt += sum(y == y_pred)

    return correct_cnt/(n_batch*batch_size)

In [151]:
def score_stagger_before_w_split(model, n_batch, batch_size, split_number):
    correct_cnt = 0
    stream = STAGGERGenerator(classification_function = 0, balance_classes = True)
    for i in range(n_batch):
        stream.generate_drift()
        for j in range(split_number):
            X, y = stream.next_sample(batch_size//split_number)
            model.partial_fit(X, y, classes=[0, 1])

            X, y = stream.next_sample(batch_size//split_number)
            y_pred = model.predict(X)
            correct_cnt += sum(y == y_pred)
    return correct_cnt/(n_batch*batch_size)

In [152]:
def score_stagger_after_w_split(model, n_batch, batch_size, split_number):
    correct_cnt = 0
    stream = STAGGERGenerator(classification_function = 0, balance_classes = True)
    for i in range(n_batch):
        stream.generate_drift()
        for j in range(split_number):
            X, y = stream.next_sample(batch_size//split_number)
            y_pred = model.predict(X)
            correct_cnt += sum(y == y_pred)
            
            X, y = stream.next_sample(batch_size//split_number)
            model.partial_fit(X, y, classes=[0, 1])
    return correct_cnt/(n_batch*batch_size)

In [153]:
%%time
for split in [1, 2, 5, 10, 20, 50, 100]:
    print('score_stagger_after_w_split', split)
    '''model = AdditiveExpertEnsembleClassifier()
    print('AdditiveExpertEnsembleClassifier :{}'.format(score_stagger_after_w_split(
        model, n_batch=120, batch_size=500, split_number=split)))
    model = DynamicWeightedMajorityClassifier()
    print('DynamicWeightedMajorityClassifier :{}'.format(score_stagger_after_w_split(
        model, n_batch=120, batch_size=500, split_number=split)))'''
    model = LearnPPNSEClassifier()
    print('LearnPPNSEClassifier :{}'.format(score_stagger_after_w_split(
        model, n_batch=120, batch_size=500, split_number=split)))

score_stagger_after_w_split 1
LearnPPNSEClassifier :0.3833166666666667
score_stagger_after_w_split 2
LearnPPNSEClassifier :0.69655
score_stagger_after_w_split 5
LearnPPNSEClassifier :0.6274666666666666
score_stagger_after_w_split 10
LearnPPNSEClassifier :0.69515
score_stagger_after_w_split 20
LearnPPNSEClassifier :0.6881
score_stagger_after_w_split 50
LearnPPNSEClassifier :0.6809166666666666
score_stagger_after_w_split 100


KeyboardInterrupt: 

In [ ]:
%%time
for split in [1, 2, 5, 10, 20, 50, 500]:
    print('score_stagger_before_w_split', split)
    """model = AdditiveExpertEnsembleClassifier()
    print('AdditiveExpertEnsembleClassifier :{}'.format(score_stagger_before_w_split(
        model, n_batch=120, batch_size=500, split_number=split)))
    model = DynamicWeightedMajorityClassifier()
    print('DynamicWeightedMajorityClassifier :{}'.format(score_stagger_before_w_split(
        model, n_batch=120, batch_size=500, split_number=split)))"""
    model = LearnPPNSEClassifier()
    print('LearnPPNSEClassifier :{}'.format(score_stagger_before_w_split(
        model, n_batch=120, batch_size=500, split_number=split)))

score_stagger_before_w_split 1
LearnPPNSEClassifier :1.0
score_stagger_before_w_split 2
LearnPPNSEClassifier :1.0
score_stagger_before_w_split 5
LearnPPNSEClassifier :0.7409666666666667
score_stagger_before_w_split 10
LearnPPNSEClassifier :0.7477
score_stagger_before_w_split 20
LearnPPNSEClassifier :0.7295333333333334
score_stagger_before_w_split 50
LearnPPNSEClassifier :0.70735
score_stagger_before_w_split 500


In [14]:
%%time
print('score_stagger_after')
model = AdditiveExpertEnsembleClassifier()
print('AdditiveExpertEnsembleClassifier :{}'.format(score_stagger_after(model, n_batch=120, batch_size=500)))
model = DynamicWeightedMajorityClassifier()
print('DynamicWeightedMajorityClassifier :{}'.format(score_stagger_after(model, n_batch=120, batch_size=500)))
model = LearnPPNSEClassifier()
print('LearnPPNSEClassifier :{}'.format(score_stagger_after(model, n_batch=120, batch_size=500)))

score_stagger_after
AdditiveExpertEnsembleClassifier :0.5677666666666666
DynamicWeightedMajorityClassifier :0.3802833333333333
LearnPPNSEClassifier :0.3889166666666667
CPU times: user 7min 4s, sys: 876 ms, total: 7min 5s
Wall time: 7min 3s


In [15]:
%%time
print('score_stagger_before')
model = AdditiveExpertEnsembleClassifier()
print('AdditiveExpertEnsembleClassifier :{}'.format(score_stagger_before(model, n_batch=120, batch_size=500)))
model = DynamicWeightedMajorityClassifier()
print('DynamicWeightedMajorityClassifier :{}'.format(score_stagger_before(model, n_batch=120, batch_size=500)))
model = LearnPPNSEClassifier()
print('LearnPPNSEClassifier :{}'.format(score_stagger_before(model, n_batch=120, batch_size=500)))

score_stagger_before
AdditiveExpertEnsembleClassifier :0.6315
DynamicWeightedMajorityClassifier :0.9791666666666666
LearnPPNSEClassifier :1.0
CPU times: user 8min 2s, sys: 1.23 s, total: 8min 3s
Wall time: 8min 1s


In [54]:
def score_sea(model, n_batch, batch_size):
    correct_cnt = 0
    for i in range(n_batch):
        stream = STAGGERGenerator(classification_function = i%3, random_state = i, balance_classes = True)
        X, y = stream.next_sample(batch_size)
        
        y_pred = model.predict(X)
        correct_cnt += sum(y == y_pred)
        model.partial_fit(X, y, classes=[0, 1])
    return correct_cnt/(n_batch*batch_size)

In [57]:
124030//500

248

In [94]:
streams = [STAGGERGenerator
               (
                   classification_function = i, random_state = i, balance_classes = True
               ) for i in range(3)]

In [96]:
streams[0]

STAGGERGenerator(balance_classes=True, classification_function=0,
                 random_state=0)

In [106]:
def score_stagger_window(model, n_batch, batch_size):
    correct_cnt = 0
    streams = [STAGGERGenerator
               (
                   classification_function = i, random_state = i, balance_classes = True
               ) for i in range(3)]
    for i in range(n_batch*batch_size):
        X, y = streams[(i//500)%3].next_sample(1)
        y_pred = model.predict(X)
        correct_cnt += sum(y == y_pred)
        if i%batch_size == 0:
            X, y = streams[(i//500)%3].next_sample(batch_size)
            model.partial_fit(X, y, classes=[0, 1])
    return correct_cnt/(n_batch*batch_size)

In [105]:
%%time
model = AdditiveExpertEnsembleClassifier(pruning='oldest')
score_stagger_window(model, n_batch=12, batch_size=500)

CPU times: user 2.1 s, sys: 0 ns, total: 2.1 s
Wall time: 2.1 s


0.5011666666666666

In [107]:
%%time
model = AdditiveExpertEnsembleClassifier(pruning='oldest')
score_stagger_window(model, n_batch=12, batch_size=500)

CPU times: user 6.34 s, sys: 7.95 ms, total: 6.35 s
Wall time: 6.36 s


0.9985

In [65]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_stagger(model, 500, 12)

CPU times: user 5.18 s, sys: 3.98 ms, total: 5.18 s
Wall time: 5.19 s


0.7363333333333333

In [72]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_stagger_window(model, 500, 12)

CPU times: user 4.27 s, sys: 8.02 ms, total: 4.28 s
Wall time: 4.28 s


0.9955

In [73]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_stagger_window(model, 200, 12)

CPU times: user 1.73 s, sys: 8.05 ms, total: 1.74 s
Wall time: 1.74 s


0.98875

In [76]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_stagger_window(model, 200, 12)

CPU times: user 1.76 s, sys: 8.06 ms, total: 1.77 s
Wall time: 1.77 s


0.99125

In [80]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_stagger_window(model, 200, 12)

CPU times: user 1.01 s, sys: 4.03 ms, total: 1.02 s
Wall time: 1.01 s


0.5004166666666666

In [67]:
5973/6000

0.9955

In [ ]:
retrain_window = 

In [55]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_sea(model, 500, 120)

CPU times: user 4.35 s, sys: 3.99 ms, total: 4.35 s
Wall time: 4.35 s


0.5936

In [47]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_stagger(model, 500, 127)

CPU times: user 50.9 s, sys: 39.8 ms, total: 51 s
Wall time: 50.9 s


0.7325354330708661

In [51]:
%%time
model = AdditiveExpertEnsembleClassifier()
score_sea(model, 200, 127)

CPU times: user 20.4 s, sys: 3.84 ms, total: 20.4 s
Wall time: 20.4 s


0.5771259842519685

In [49]:
500*120

60000

In [ ]:
%%time
for estimator in [5, 15, 25]:
    for batch in [50, 120, 377]:
        print('estimator : {}, batch: {}'.format(estimator, batch))
        score_elec(AdditiveExpertEnsembleClassifier(n_estimators=estimator), batch)

estimator : 5, batch: 50
Learn++.NSE classifier accuracy: 0.724708686440678
estimator : 5, batch: 120
Learn++.NSE classifier accuracy: 0.7178893008474576
estimator : 5, batch: 377
Learn++.NSE classifier accuracy: 0.692134533898305
estimator : 15, batch: 50
Learn++.NSE classifier accuracy: 0.724752824858757
estimator : 15, batch: 120
Learn++.NSE classifier accuracy: 0.7179113700564972
estimator : 15, batch: 377
Learn++.NSE classifier accuracy: 0.692134533898305
estimator : 25, batch: 50


In [29]:
%%time
for estimator in [5]:
    for batch in [10, 20, 30, 40]:
        print('estimator : {}, batch: {}'.format(estimator, batch))
        score_elec(AdditiveExpertEnsembleClassifier(n_estimators=estimator), batch)

estimator : 5, batch: 10
Learn++.NSE classifier accuracy: 0.7290342514124294
estimator : 5, batch: 20
Learn++.NSE classifier accuracy: 0.728129413841808
estimator : 5, batch: 30
Learn++.NSE classifier accuracy: 0.7273128531073446
estimator : 5, batch: 40
Learn++.NSE classifier accuracy: 0.725150070621469
CPU times: user 3min 54s, sys: 19.6 ms, total: 3min 54s
Wall time: 3min 54s


In [30]:
%%time
for estimator in [5]:
    for batch in [1]:
        print('estimator : {}, batch: {}'.format(estimator, batch))
        score_elec(AdditiveExpertEnsembleClassifier(n_estimators=estimator), batch)

estimator : 5, batch: 1
Learn++.NSE classifier accuracy: 0.7321239406779662
CPU times: user 2min 53s, sys: 20 ms, total: 2min 53s
Wall time: 2min 53s


In [32]:
# Imports
from skmultiflow.data.stagger_generator import STAGGERGenerator
# Setting up the stream
stream = STAGGERGenerator(classification_function = 2, random_state = 112, balance_classes = False)
# Retrieving one sample
stream.next_sample()
stream.next_sample(10)
stream.n_remaining_samples()
stream.has_more_samples()

True

In [ ]:
def 

In [33]:
stream.next_sample(10)

(array([[2., 0., 1.],
        [2., 0., 0.],
        [1., 0., 0.],
        [0., 2., 1.],
        [2., 2., 0.],
        [2., 1., 2.],
        [0., 2., 0.],
        [1., 0., 2.],
        [1., 2., 2.],
        [1., 0., 1.]]),
 array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1]))